# HISSO Logging CLI ÃƒÂ¢Ã¢â€šÂ¬Ã¢â‚¬Å“ GPU Colab Run

Execute this notebook in Google Colab (GPU runtime) after the refreshed `psann` package is published to PyPI. It installs the released wheel, prepares synthetic datasets/configs, and runs the logging CLI on CUDA to produce shareable metrics, event logs, and checkpoints.

## 0. Release checklist

1. Publish the new `psann` version to PyPI.
2. Update `PSANN_VERSION` below to match the release (e.g., `"0.10.4"`).
3. (Optional) Adjust run tags/epochs to fit the available GPU budget.
4. Switch Colab to **Runtime ÃƒÂ¢Ã¢â‚¬Â Ã¢â‚¬â„¢ Change runtime type ÃƒÂ¢Ã¢â‚¬Â Ã¢â‚¬â„¢ GPU** before executing the remaining cells.

In [1]:
#@title Configure versions and run metadata
PSANN_PACKAGE_SPEC = ""  #@param {type:"string"}  # e.g. "psann==0.10.10" or "git+https://github.com/Nickm1128/psann.git@main"
TORCH_VERSION = "2.5.1+cu121"  #@param {type:"string"}
TORCHVISION_VERSION = "0.20.1+cu121"  #@param {type:"string"}
TORCHAUDIO_VERSION = "2.5.1+cu121"  #@param {type:"string"}
TORCH_INDEX_URL = "https://download.pytorch.org/whl/cu121"  #@param {type:"string"}

RUN_TAG = "colab_gpu"  #@param {type:"string"}
OUTPUT_ROOT = "/content/hisso_runs"  #@param {type:"string"}
CONFIG_ROOT = "/content/hisso_configs"  #@param {type:"string"}
DATA_ROOT = "/content/hisso_data"  #@param {type:"string"}

DENSE_EPOCHS = 8  #@param {type:"integer"}
WAVE_EPOCHS = 10  #@param {type:"integer"}
USE_MIXED_PRECISION = True  #@param {type:"boolean"}
INSTALL_DEPENDENCIES = False  #@param {type:"boolean"}
FORCE_DEVICE = "auto"  #@param ["auto", "cpu", "cuda"]


In [2]:
# Install CUDA-enabled PyTorch wheels and the published psann package.
import importlib
import subprocess
import sys

def _pip_install(*args: str) -> None:
    cmd = [sys.executable, "-m", "pip", "install", "-q", *args]
    print("$", " ".join(cmd))
    subprocess.check_call(cmd)

if INSTALL_DEPENDENCIES:
    _pip_install(
        f"torch=={TORCH_VERSION}",
        f"torchvision=={TORCHVISION_VERSION}",
        f"torchaudio=={TORCHAUDIO_VERSION}",
        "--extra-index-url",
        TORCH_INDEX_URL,
    )
    _pip_install("pyyaml>=6.0")
    if PSANN_PACKAGE_SPEC:
        _pip_install(PSANN_PACKAGE_SPEC)
    else:
        _pip_install("psann")
else:
    try:
        importlib.import_module("yaml")
    except ModuleNotFoundError:
        _pip_install("pyyaml>=6.0")
    try:
        importlib.import_module("psann.scripts.hisso_log_run")
        print("Using existing psann installation.")
    except ModuleNotFoundError:
        raise RuntimeError("psann installation is missing scripts. Set INSTALL_DEPENDENCIES=True and provide PSANN_PACKAGE_SPEC (PyPI version or git URL) that includes psann.scripts.")


Using existing psann installation.


In [3]:
# Verify CUDA availability and pick a device.
import torch

def _resolve_device(force: str) -> str:
    force = force.lower()
    if force == "auto":
        return "cuda" if torch.cuda.is_available() else "cpu"
    if force.startswith("cuda") and not torch.cuda.is_available():
        print("Requested CUDA but no device is available; falling back to CPU.")
        return "cpu"
    return force

TARGET_DEVICE = _resolve_device(FORCE_DEVICE)
USE_AMP = bool(USE_MIXED_PRECISION and TARGET_DEVICE.startswith("cuda") and torch.cuda.is_available())

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device name: {torch.cuda.get_device_name(0)}")
print(f"Using device: {TARGET_DEVICE}")
print(f"Mixed precision: {USE_AMP}")
print(f"Torch version: {torch.__version__}, CUDA runtime: {torch.version.cuda}")


CUDA available: False
Using device: cpu
Mixed precision: False
Torch version: 2.7.1+cu118, CUDA runtime: 11.8


In [4]:
# Prepare directories for configs, data, and run artefacts.
from pathlib import Path

for path in (Path(OUTPUT_ROOT), Path(CONFIG_ROOT), Path(DATA_ROOT)):
    path.mkdir(parents=True, exist_ok=True)

print("Output root:", OUTPUT_ROOT)
print("Config root:", CONFIG_ROOT)
print("Data root:", DATA_ROOT)

Output root: /content/hisso_runs
Config root: /content/hisso_configs
Data root: /content/hisso_data


In [5]:
# Synthesise a WaveResNet-style dataset (channels-first sequences).
import numpy as np

rng = np.random.default_rng(seed=42)

def _make_split(num_samples: int, channels: int, steps: int, targets: int):
    X = rng.normal(size=(num_samples, channels, steps)).astype(np.float32)
    y = rng.normal(size=(num_samples, targets)).astype(np.float32)
    prices = np.clip(1.0 + rng.normal(scale=0.01, size=(num_samples, targets)), 1e-3, None).astype(np.float32)
    return X, y, prices

train_X, train_y, train_prices = _make_split(320, 4, 128, 3)
val_X, val_y, val_prices = _make_split(96, 4, 128, 3)
test_X, test_y, test_prices = _make_split(96, 4, 128, 3)

wave_npz_path = Path(DATA_ROOT) / "wave_resnet_synth.npz"
np.savez_compressed(
    wave_npz_path,
    X_train=train_X,
    y_train=train_y,
    prices_train=train_prices,
    X_val=val_X,
    y_val=val_y,
    prices_val=val_prices,
    X_test=test_X,
    y_test=test_y,
    prices_test=test_prices,
)
print("Synthetic dataset written to", wave_npz_path)

Synthetic dataset written to \content\hisso_data\wave_resnet_synth.npz


In [10]:
# Emit HISSO config files targeting the selected device.
import textwrap

dense_config_path = Path(CONFIG_ROOT) / "dense_run.yaml"
dense_config = f"""
estimator:
  target: psann.PSANNRegressor
  params:
    hidden_layers: 2
    hidden_units: 64
    epochs: {DENSE_EPOCHS}
    batch_size: 128
    lr: 0.0008
    random_state: 7
    device: {TARGET_DEVICE}
hisso:
  enabled: true
  window: 32
  primary_transform: softmax
  transition_penalty: 0.0
  mixed_precision: {str(USE_AMP).lower()}
data:
  loader: psann.scripts.hisso_log_run.toy_hisso_dataset
  kwargs:
    steps: 256
    features: 6
    seed: 13
    train_fraction: 0.6
    val_fraction: 0.2
training:
  verbose: 1
evaluation:
  portfolio_prices_key: prices_test
  trans_cost: 0.001
"""

dense_config_path.write_text(textwrap.dedent(dense_config).strip() + "", encoding="utf-8")

wave_config_path = Path(CONFIG_ROOT) / "wave_resnet_run.yaml"
wave_config = f"""
estimator:
  target: psann.WaveResNetRegressor
  params:
    hidden_layers: 3
    hidden_units: 64
    epochs: {WAVE_EPOCHS}
    batch_size: 24
    lr: 0.0005
    random_state: 11
    device: {TARGET_DEVICE}
    preserve_shape: false
    data_format: channels_first
    w0_warmup_epochs: 5
    progressive_depth_initial: 2
    progressive_depth_interval: 5
hisso:
  enabled: true
  window: 32
  primary_transform: softmax
  transition_penalty: 0.01
  mixed_precision: {str(USE_AMP).lower()}
  supervised:
    y_key: y_train
    epochs: 2
    batch_size: 64
data:
  npz: {wave_npz_path}
training:
  verbose: 1
evaluation:
  portfolio_prices_key: prices_test
  trans_cost: 0.001
"""

wave_config_path.write_text(textwrap.dedent(wave_config).strip() + "", encoding="utf-8")

print("Dense config:", dense_config_path)
print("WaveResNet config:", wave_config_path)


Dense config: \content\hisso_configs\dense_run.yaml
WaveResNet config: \content\hisso_configs\wave_resnet_run.yaml


In [11]:
# Run the dense HISSO config.
import datetime as _dt

timestamp = _dt.datetime.now(_dt.timezone.utc).strftime("%Y%m%d_%H%M%S")
dense_run_name = f"dense_{TARGET_DEVICE}_{RUN_TAG}_{timestamp}"
print("Launching dense run:", dense_run_name)

cmd = [
    sys.executable,
    "-m",
    "psann.scripts.hisso_log_run",
    "--config",
    str(dense_config_path),
    "--output-dir",
    str(Path(OUTPUT_ROOT) / "dense"),
    "--device",
    TARGET_DEVICE,
    "--run-name",
    dense_run_name,
    "--seed",
    "7",
    "--verbose",
]
print("$", " ".join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)
if result.stdout:
    print(result.stdout)
if result.stderr:
    print(result.stderr, file=sys.stderr)
if result.returncode != 0:
    raise RuntimeError(f"Dense run failed with exit code {result.returncode}")


Launching dense run: dense_cpu_colab_gpu_20251030_204233


0

In [12]:
# Run the WaveResNet HISSO config.
import datetime as _dt

timestamp_wave = _dt.datetime.now(_dt.timezone.utc).strftime("%Y%m%d_%H%M%S")
wave_run_name = f"wave_resnet_{TARGET_DEVICE}_{RUN_TAG}_{timestamp_wave}"
print("Launching wave-resnet run:", wave_run_name)

cmd = [
    sys.executable,
    "-m",
    "psann.scripts.hisso_log_run",
    "--config",
    str(wave_config_path),
    "--output-dir",
    str(Path(OUTPUT_ROOT) / "wave_resnet"),
    "--device",
    TARGET_DEVICE,
    "--run-name",
    wave_run_name,
    "--seed",
    "11",
    "--verbose",
]
print("$", " ".join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)
if result.stdout:
    print(result.stdout)
if result.stderr:
    print(result.stderr, file=sys.stderr)
if result.returncode != 0:
    raise RuntimeError(f"WaveResNet run failed with exit code {result.returncode}")


Launching wave-resnet run: wave_resnet_cpu_colab_gpu_20251030_204242


0

In [13]:
# Summarise generated artefacts.
import json
from pprint import pprint

def _collect_latest(run_dir: Path) -> Path:
    candidates = sorted(run_dir.glob("*/metrics.json"), key=lambda p: p.stat().st_mtime)
    if not candidates:
        raise FileNotFoundError(f"No metrics.json under {run_dir}")
    return candidates[-1]

dense_metrics_path = _collect_latest(Path(OUTPUT_ROOT) / "dense")
wave_metrics_path = _collect_latest(Path(OUTPUT_ROOT) / "wave_resnet")

print("Dense metrics:", dense_metrics_path)
with dense_metrics_path.open() as fh:
    dense_metrics = json.load(fh)
pprint(dense_metrics)

print("\nWaveResNet metrics:", wave_metrics_path)
with wave_metrics_path.open() as fh:
    wave_metrics = json.load(fh)
pprint(wave_metrics)


Dense metrics: \content\hisso_runs\dense\dense_cpu_colab_gpu_20251030_204233\metrics.json
{'device': 'cpu',
 'duration_seconds': 1.9047843001317233,
 'hisso': {'best_epoch': 6,
           'episodes': 256,
           'profile': {'amp_dtype': None,
                       'amp_enabled': False,
                       'batch_episodes': 32,
                       'dataset_bytes': 3672,
                       'dataset_numpy_to_tensor_time_s': 4.220008850097656e-05,
                       'dataset_transfer_batches': 1,
                       'dataset_transfer_time_s': 7.4999406933784485e-06,
                       'device': 'cpu',
                       'episode_length': 32,
                       'episode_time_s_total': 0.5841135994996876,
                       'episode_view_is_shared': True,
                       'episodes_sampled': 256,
                       'epochs': 8,
                       'total_time_s': 0.5872145998291671},
           'reward_mean': -0.11111112657818012,
          

## 7. Export artefacts

Optionally archive the run directory for download (uncomment the cell below when needed).

```python
# import shutil
# shutil.make_archive("hisso_runs", "zip", OUTPUT_ROOT)
```

Mounting Google Drive (`from google.colab import drive`) is also recommended if you want the metrics and checkpoints to persist automatically.